##Importation and Installation

In [ ]:
!pip install transformers datasets huggingface_hub torch gradio accelerate -q

import os
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

##Login in Hugging Face to authenticate and use StarCoder Model

In [ ]:
from google.colab import userdata

HF_TOKEN = userdata.get("colabnb")

if HF_TOKEN is None:
    raise ValueError("HF token not found in environment variable 'colabnb'.")

login(token=HF_TOKEN)

##Setting up the Model by Tokenizing, Loading  Language Models locally and caching

In [ ]:
MODEL_NAME = "bigcode/starcoderbase-1b"

tokenizer = None
model = None
device = "cuda" if torch.cuda.is_available() else "cpu"

def load_model():
    global tokenizer, model
    if tokenizer is None or model is None:
        print("Loading model...")
        torch_dtype = torch.float16 if device == "cuda" else torch.float32

        tokenizer_local = AutoTokenizer.from_pretrained(
            MODEL_NAME,
            token=HF_TOKEN
        )

        model_local = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            token=HF_TOKEN,
            torch_dtype=torch_dtype
        )

        model_local.to(device)
        model_local.eval()

        tokenizer = tokenizer_local
        model = model_local

    return tokenizer, model

##Forming Snippets to guide our Model

In [ ]:
SNIPPETS = {
    "python": {
        "basics": {
            "beginner": [
                "x = 5\nprint(x)",
                "name = input('Enter your name: ')\nprint('Hello,', name)"
            ],
            "intermediate": [
                "numbers = [1, 2, 3, 4]\nsquared = [n**2 for n in numbers]\nprint(squared)"
            ],
            "advanced": [
                "def memoized_fib(n, cache={}):\n    if n in cache:\n        return cache[n]\n    if n <= 1:\n        return n\n    cache[n] = memoized_fib(n-1) + memoized_fib(n-2)\n    return cache[n]"
            ],
        },
        "loops": {
            "beginner": [
                "for i in range(5):\n    print(i)"
            ],
            "intermediate": [
                "items = ['apple', 'banana', 'cherry']\nfor idx, item in enumerate(items):\n    print(idx, item)"
            ],
            "advanced": [
                "matrix = [[1,2,3],[4,5,6]]\nfor row in matrix:\n    for value in row:\n        print(value)"
            ],
        },
    },
    "java": {
        "basics": {
            "beginner": [
                "public class Main {\n  public static void main(String[] args) {\n    int x = 5;\n    System.out.println(x);\n  }\n}"
            ],
            "intermediate": [
                "public class Main {\n  public static void main(String[] args) {\n    int[] nums = {1,2,3};\n    for (int n : nums) {\n      System.out.println(n);\n    }\n  }\n}"
            ],
            "advanced": [
                "class Person {\n  private String name;\n  public Person(String name) { this.name = name; }\n  public String getName() { return name; }\n}"
            ],
        },
        "loops": {
            "beginner": [
                "public class Main {\n  public static void main(String[] args) {\n    for (int i = 0; i < 5; i++) {\n      System.out.println(i);\n    }\n  }\n}"
            ],
            "intermediate": [
                "public class Main {\n  public static void main(String[] args) {\n    String[] items = {\"apple\", \"banana\", \"cherry\"};\n    for (String item : items) {\n      System.out.println(item);\n    }\n  }\n}"
                ],
            "advanced": [
                "public class Main {\n  public static void main(String[] args) {\n    int[][] matrix = {{1,2,3},{4,5,6}};\n    for (int[] row : matrix) {\n      for (int val : row) {\n        System.out.println(val);\n      }\n    }\n  }\n}"
            ],
        },
    },
    "c": {
        "basics": {
            "beginner": [
                "#include <stdio.h>\nint main() {\n  int x = 5;\n  printf(\"%d\", x);\n  return 0;\n}"
            ],
            "intermediate": [
                "#include <stdio.h>\nint main() {\n  int arr[3] = {1,2,3};\n  for (int i = 0; i < 3; i++) {\n    printf(\"%d\\n\", arr[i]);\n  }\n  return 0;\n}"
            ],
            "advanced": [
                "#include <stdio.h>\nvoid swap(int *a, int *b) {\n  int temp = *a;\n  *a = *b;\n  *b = temp;\n}"
            ],
        },
        "loops": {
            "beginner": [
                "#include <stdio.h>\nint main() {\n  for (int i = 0; i < 5; i++) {\n    printf(\"%d\\n\", i);\n  }\n  return 0;\n}"
            ],
            "intermediate": [
                "#include <stdio.h>\nint main() {\n  int nums[] = {1,2,3};\n  int size = sizeof(nums)/sizeof(nums[0]);\n  int i = 0;\n  while (i < size) {\n    printf(\"%d\\n\", nums[i]);\n    i++;\n  }\n  return 0;\n}"
            ],
            "advanced": [
                "#include <stdio.h>\nint main() {\n  int matrix[2][3] = {{1,2,3},{4,5,6}};\n  for (int i = 0; i < 2; i++) {\n    for (int j = 0; j < 3; j++) {\n      printf(\"%d\\n\", matrix[i][j]);\n    }\n  }\n  return 0;\n}"
            ],
        },
    },
}

def get_snippets(language, module, difficulty, max_snippets=2):
    language = language.lower()
    module = module.lower()
    difficulty = difficulty.lower()
    return SNIPPETS.get(language, {}).get(module, {}).get(difficulty, [])[:max_snippets]

##Using Prompt Engineering to generate required outputs

In [ ]:
def compose_prompt(language, module, difficulty):
    lang = language.lower()
    mod = module.lower()
    diff = difficulty.lower()

    if diff == "beginner":
        style = (
            "Explain like I'm totally new to programming. "
            "Use very simple words, very small steps, 1–2 tiny examples, "
            "and one basic quiz question with answer."
        )
    elif diff == "intermediate":
        style = (
            "Assume I know the basics. "
            "Explain with more depth, use around 2 examples, "
            "and then give a small coding challenge and outline the solution."
        )
    else:  # advanced
        style = (
            "Assume I already know the fundamentals very well. "
            "Explain in detail with edge cases and performance considerations. "
            "Give at least one tricky challenge and then provide a clear, well-commented solution."
        )

    prompt = f"""
You are a patient, structured AI programming tutor.

Teach the topic: {language} - {module}
Difficulty level: {difficulty}

{style}

Very important formatting rules (follow exactly):

1. You MUST output only these 4 sections, in order, with these exact headings:
   1. Explanation
   2. Examples
   3. Quiz
   4. Challenge

2. Do NOT add any other sections like "Additional resources", "References", or links.
3. Do NOT output external URLs.
4. Use Markdown formatting with code blocks where needed.

For the sections:

- 1. Explanation: Explain the concept clearly for the given difficulty.
- 2. Examples: Give at least 1–2 code examples in {language}.
- 3. Quiz: 2–4 short conceptual questions with their answers.
- 4. Challenge: 1 coding task appropriate to the difficulty.
  For Intermediate or Advanced, also provide a suggested solution.

Now write the full lesson in Markdown, starting exactly with:

1. Explanation
"""

    snippets = get_snippets(language, module, difficulty)
    if snippets:
        prompt += "\nHere are some example code snippets in this language and difficulty. " \
                  "Use them only as inspiration, do not just repeat them:\n"
        for s in snippets:
            prompt += f"\n`{lang}\n{s}\n```\n"

    prompt += "\nRemember: do NOT include 'Additional resources', links, or any extra sections.\n"
    return prompt

##Adding sanity checks to avoid garbage or hallucinated outputs

In [ ]:
def sanity_fix_lesson(text: str) -> str:
    stripped = text.strip()

    if len(stripped) < 80:
        return (
            "1. Explanation\n\n"
            "The model returned a very short answer. "
            "Please click Start Learning again to regenerate the lesson.\n"
        )

    lowered = stripped.lower()
    if ("additional resources" in lowered or "references" in lowered) and \
       ("1. explanation" not in lowered and "2. examples" not in lowered):
        return (
            "1. Explanation\n\n"
            "The model generated an invalid lesson structure. "
            "Please try again; if the issue repeats, reduce difficulty or change topic.\n"
        )

    sections = ["1. Explanation", "2. Examples", "3. Quiz", "4. Challenge"]
    missing = [s for s in sections if s not in stripped]

    if missing:
        patched = stripped
        if "1. Explanation" not in patched:
            patched = "1. Explanation\n\n" + patched
        if "2. Examples" not in patched:
            patched += "\n\n2. Examples\n(Examples were not generated. Try regenerating if needed.)"
        if "3. Quiz" not in patched:
            patched += "\n\n3. Quiz\n(Quiz questions were not generated. Try regenerating if needed.)"
        if "4. Challenge" not in patched:
            patched += "\n\n4. Challenge\n(Challenge was not generated. Try regenerating if needed.)"
        return patched

    return stripped

##Adding repetition filters to avoid repetitive outputs

In [ ]:
def generate_lesson(language, module, difficulty):
    tok, mdl = load_model()
    prompt = compose_prompt(language, module, difficulty)

    diff = difficulty.lower()

    if diff == "beginner":
        max_tokens = 220
        temperature = 0.55
    elif diff == "intermediate":
        max_tokens = 300
        temperature = 0.5
    else:  # advanced
        max_tokens = 340
        temperature = 0.48

    if language.lower() in ["java", "c"]:
        max_tokens += 40

    inputs = tok(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=768,
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = mdl.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            use_cache=True,
            pad_token_id=tok.eos_token_id,
            no_repeat_ngram_size=4,
            repetition_penalty=1.1,
        )

    text = tok.decode(outputs[0], skip_special_tokens=True)

    if text.startswith(prompt):
        text = text[len(prompt):].strip()
    elif prompt in text:
        text = text.split(prompt, 1)[-1].strip()

    lesson = sanity_fix_lesson(text)
    return lesson

##Integrating Gradio Web-App UI to generate lesson

In [ ]:
def ui_generate(language, module, difficulty):
    lesson = generate_lesson(language, module, difficulty)
    header = f"### Lesson: {language} – {module} ({difficulty})\n\n"
    return header + lesson

load_model()

with gr.Blocks() as demo:
    gr.Markdown("# AI Programming Tutor")
    gr.Markdown(
        "Choose a language, topic, and difficulty, then click "
        "Start Learning to generate a custom lesson powered by StarCoder."
    )

    with gr.Row():
        lang = gr.Dropdown(
            ["Python", "Java", "C"],
            label="Language",
            value="Python"
        )
        mod = gr.Dropdown(
            ["basics", "loops", "functions", "oop", "pointers"],
            label="Module",
            value="basics"
        )
        diff = gr.Dropdown(
            ["Beginner", "Intermediate", "Advanced"],
            label="Difficulty",
            value="Beginner"
        )

    start_btn = gr.Button("Start Learning")
    output = gr.Markdown(label="Lesson")

    start_btn.click(fn=ui_generate, inputs=[lang, mod, diff], outputs=output)

demo.launch(share=True)

Loading model...


tokenizer_config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6691dc00d75232f65b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
